# Import Package

In [ ]:
import time
import random
import os
import requests
from bs4 import BeautifulSoup

# THSRC Image Downloader

In [1]:
class THSRCImageDownloader:
    def __init__(self, url, download_path):
        self.url = url
        self.download_path = download_path
        self.headers = { 
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
            "Cache-Control": "max-age=0",
            "Connection": "keep-alive",
            "Host": "irs.thsrc.com.tw",
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": "Windows",
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Site": "none",
            "Sec-Fetch-User": "?1",
            "Upgrade-Insecure-Requests": "1",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
        }
        
        if not os.path.exists(download_path):
            os.mkdir(download_path)
    
    def get_validation_code_link(self):
        r = requests.get(self.url, timeout=5, headers=self.headers)
        if r.status_code != requests.codes.ok:
            print("CAN NOT LOADING")
            return []
        soup = BeautifulSoup(r.text, 'html.parser')
        attr = {"class": "captcha-img"}
        img_tags = soup.find_all('img', attrs=attr)
        for tag in img_tags:
            link_str = self.url + tag["src"][6:]
            print(link_str)
        return link_str
    
    def download_img(self, img_index, link):
        img = requests.get(link, headers=self.headers)
        img = img.content
        with open(self.download_path + str(img_index) + '.jpg', 'wb') as pic_out:
            pic_out.write(img)

        print('img_index:', img_index, 'OK!')

# Batch Downloader

In [ ]:
class THSRCImageBatchDownloader(THSRCImageDownloader):
    def __init__(self, url, download_path, img_num):
        super().__init__(url, download_path)
        self.img_num = img_num
        
    def download_images(self):
        for i in range(1, self.img_num + 1):
            time.sleep(random.randint(1, 2))
            link = self.get_validation_code_link()
            self.download_img(i, link)

# Run

In [ ]:
url = 'http://irs.thsrc.com.tw/IMINT/'
download_path = ""
img_num = 5000

batch_downloader = THSRCImageBatchDownloader(url, download_path, img_num)
batch_downloader.download_images()